In [48]:
import faiss
import numpy as np
import time
import networkx as nx

res = faiss.StandardGpuResources()

In [ ]:
nx.erdos_renyi_graph(n, p)

In [60]:
def load_state(path='./data/states/12340689.txt'):
    with open(path, 'r') as f:
        d = f.read().split("\n")[:-1]
        
    positions, forces, ticker = [], [], 0
    pos, forc = [], []
    for item in d:
        if ticker < 3:
            pos.append(float(item))
        else:
            forc.append(float(item))
        ticker += 1
        if ticker > 5:
            ticker = 0
            positions.append(pos), forces.append(forc)
            pos, forc = [], []
            
    return np.array(positions), np.array(forces)

def build_nn_graph(p, k=6, g=None, use_gpu=True):
    if g is None:
        g = nx.Graph()
#     t1 = time.time()
    p = p.astype("float32")
    index_flat = faiss.IndexFlatL2(3)
    if use_gpu:
        index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
    index_flat.add(p)
    _, I = index_flat.search(p, k+1)
    I = I[:, 1:]
#     print(time.time() - t1)
#     edges = set()
    for i in range(len(p)):
        row = I[i, :]
        for item in row:
            edge = (i, item)
            edge = (min(edge), max(edge))
            if edge[0] != edge[1]:
                g.add_edge(edge[0], edge[1])
    return g
    
    
p, f = load_state()
I = build_nn_graph(p)

In [44]:
p = np.random.random((10000, 3)).astype('float32')

t1 = time.time()
index_flat = faiss.IndexFlatL2(3) 
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
# index = faiss.IndexIVFFlat(3)
gpu_index_flat.add(p)
_, I = gpu_index_flat.search(p, 7)
# print(I[:, 1:])
print(time.time() - t1)

0.006889820098876953


In [2]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.


64

In [20]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])  

True
100000
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.175174  7.2076287 7.251163 ]
 [0.        6.323565  6.684582  6.799944 ]
 [0.        5.7964087 6.3917365 7.2815127]
 [0.        7.277905  7.5279875 7.6628447]
 [0.        6.763804  7.295122  7.368814 ]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


array([[-0.2417869 , -0.1045786 , -0.3448408 ],
       [-0.2781265 ,  0.4904646 , -0.09036557],
       [ 0.1403343 ,  0.01452989, -0.2460732 ],
       [ 0.02065244, -0.4862413 , -0.1893059 ],
       [-0.07791606, -0.2150235 ,  0.4538759 ]])